In [41]:
import pandas as pd
from pathlib import Path
import os
from pandarallel import pandarallel
#os.chdir('../')
import datetime
import numpy as np
import re

In [2]:
pandarallel.initialize(progress_bar = True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
def ImputeTimePeriod(df, time_period_months):
    t = time_period_months
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['year'] = df['created_at'].parallel_apply(lambda x: x.year)
    df['time_period'] = df['created_at'].parallel_apply(lambda x: datetime.date(x.year, int(t*(x.month/t if x.month%t == 0 else np.ceil(x.month/t))-(t-1)), 1))
    df['time_period'] = pd.to_datetime(df['time_period'])
    return df 

In [4]:
time_period_months = 6

In [6]:
indir = Path('drive/output/derived/contributor_stats/contributor_data')
indir_linked = Path('drive/output/scrape/link_issue_pull_request')
indir_data = Path('drive/output/derived/data_export')
df_contributors = pd.read_parquet(indir / 'major_contributors_major_months6_window1828D_samplefull.parquet')
df_issue = pd.read_parquet(indir_data / 'df_issue.parquet')
df_pr = pd.read_parquet(indir_data / 'df_pr.parquet')

In [7]:
df_pr_commits = pd.read_parquet(indir_data / 'df_pr_commits.parquet')

In [8]:
selected_repos = df_issue[['repo_name']].drop_duplicates().sample(100, random_state = 1234)['repo_name'].tolist()

In [ ]:
df_issue_selected = df_issue[(df_issue['repo_name'].isin(selected_repos)) & (df_issue['created_at']>'2015-01-01')]
df_issue_selected = ImputeTimePeriod(df_issue_selected, 6)

df_pr_selected = df_pr[(df_pr['repo_name'].isin(selected_repos)) & (df_pr['created_at']>'2015-01-01')]
df_pr_selected = ImputeTimePeriod(df_pr_selected, 6)

df_pr_commits_selected = df_pr_commits[(df_pr_commits['repo_name'].isin(selected_repos))]
df_pr_commits_selected['created_at'] = pd.to_datetime(df_pr_commits_selected['commit time'], unit = 's')
df_pr_commits_selected = df_pr_commits_selected[df_pr_commits_selected['created_at']>'2015-01-01']
df_pr_commits_selected = ImputeTimePeriod(df_pr_commits_selected, 6)

In [18]:
df_opened_issues = df_issue_selected.query('issue_action == "opened"')[['time_period','created_at','repo_name','actor_id','issue_title','issue_body','issue_number']]\
    .sort_values('created_at')\
    .drop_duplicates(['repo_name','actor_id','issue_number'])

In [19]:
df_actor_opened_issues = pd.DataFrame(df_opened_issues[['time_period','repo_name','actor_id']].value_counts()).reset_index().rename({'count':'actor_issue_count'}, axis = 1)
df_actor_opened_issues['repo_issue_count'] = df_actor_opened_issues.groupby(['repo_name','time_period'])['actor_issue_count'].transform('sum')
df_actor_opened_issues['pct_issue_count'] = 100*df_actor_opened_issues['actor_issue_count']/df_actor_opened_issues['repo_issue_count']
df_actor_opened_issues = df_actor_opened_issues.sort_values(['repo_name','time_period','actor_issue_count'], ascending = [True, True, False])\
    .drop('actor_issue_count', axis = 1)

In [20]:
# at some point do this calculation for non own issue comments

In [21]:
df_issue_comments = df_issue_selected.query('type == "IssueCommentEvent"')[['time_period','created_at','repo_name','actor_id','issue_number','issue_comment_id']]\
    .sort_values('created_at')\
    .drop_duplicates(['repo_name','actor_id','issue_comment_id'])

In [22]:
df_actor_issue_comments = pd.DataFrame(df_issue_comments[['time_period','repo_name','actor_id']].value_counts()).reset_index().rename({'count':'actor_ic_count'}, axis = 1)
df_actor_issue_comments['repo_ic_count'] = df_actor_issue_comments.groupby(['repo_name','time_period'])['actor_ic_count'].transform('sum')
df_actor_issue_comments['pct_ic_count'] = 100*df_actor_issue_comments['actor_ic_count']/df_actor_issue_comments['repo_ic_count']
df_actor_issue_comments = df_actor_issue_comments.sort_values(['repo_name','time_period','actor_ic_count'], ascending = [True, True, False])\
    .drop('actor_ic_count', axis = 1)

In [23]:
df_actor_issue_comments_wt = df_issue_comments.groupby(['time_period','repo_name','actor_id','issue_number'])['issue_comment_id'].count().reset_index()\
    .rename({'issue_comment_id':'ic_count'}, axis = 1)
df_actor_issue_comments_wt['repo_ic_count'] = df_actor_issue_comments_wt.groupby(['time_period','repo_name','issue_number'])['ic_count'].transform('sum')
df_actor_issue_comments_wt['pct_ic_count'] = df_actor_issue_comments_wt['ic_count']/df_actor_issue_comments_wt['repo_ic_count']
df_actor_issue_comments_wt = df_actor_issue_comments_wt.groupby(['repo_name','time_period','actor_id']).agg({'pct_ic_count':'mean'}).reset_index().sort_values(
    ['repo_name','time_period','pct_ic_count'], ascending = [True, True, False])
df_actor_issue_comments_wt['sum_ic_count'] = df_actor_issue_comments_wt.groupby(['repo_name','time_period'])['pct_ic_count'].transform('sum')
df_actor_issue_comments_wt['pct_ic_count_wt_issue'] = df_actor_issue_comments_wt['pct_ic_count']/df_actor_issue_comments_wt['sum_ic_count']*100
df_actor_issue_comments_wt = df_actor_issue_comments_wt.drop(['pct_ic_count','sum_ic_count'], axis = 1)

In [25]:
df_pr_linked = pd.read_parquet(indir_linked / 'linked_pull_request_to_issue.parquet')
df_issue_linked = pd.read_parquet(indir_linked / 'linked_issue_to_pull_request.parquet')

In [190]:
df_pr_linked['issue_number'] = df_pr_linked['issue_link'].apply(
    lambda x: x.split("issues/")[-1].split("pull/")[-1].split("#")[0].split("%")[0].split("?")[0].split("&")[0].split("/")[0].split(":")[0].split(",")[0].split("-")[0]
        if x != 'missing' and 'Connection' not in x else np.nan)
df_pr_linked_data = df_pr_linked[['repo_name','pr_number', 'issue_number']].dropna()

df_issue_linked['pr_number'] = df_issue_linked['linked_pull_request'].apply(
    lambda x: x.split("issues/")[-1].split("pull/")[-1].split("#")[0].split("%")[0].split("?")[0].split("&")[0].split("/")[0].split(":")[0].split(",")[0].split("-")[0]
        if x != 'list index out of range' and 'Connection' not in x else np.nan)
df_issue_linked_data = df_issue_linked[['repo_name','pr_number', 'issue_number']].dropna()

In [110]:
df_issue_linked[(df_issue_linked['pr_number'] == 357.0) & (df_pr_linked['repo_name'] == 'twitter/commons')]

/tmp/user/20506/ipykernel_1137298/1187916771.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_issue_linked[(df_issue_linked['pr_number'] == 357.0) & (df_pr_linked['repo_name'] == 'twitter/commons')]


,Unnamed: 0,repo_name,issue_number,linked_pull_request,pr_number


In [191]:
df_pr_linked_data['issue_number'] = pd.to_numeric(df_pr_linked_data['issue_number'], errors = 'coerce')
df_pr_linked_data['pr_number'] = pd.to_numeric(df_pr_linked_data['pr_number'], errors = 'coerce')
df_issue_linked_data['issue_number'] = pd.to_numeric(df_issue_linked_data['issue_number'], errors = 'coerce')
df_issue_linked_data['pr_number'] = pd.to_numeric(df_issue_linked_data['pr_number'], errors = 'coerce')

In [192]:
df_linked_issue_pr = pd.merge(df_pr_linked_data, df_issue_linked_data)
df_linked_issue_pr_uq = df_linked_issue_pr.drop_duplicates().query('issue_number < pr_number').dropna()

In [200]:
df_linked_issue_pr_uq[df_linked_issue_pr_uq['repo_name'].isin(selected_repos)]
#df_issue_selected.query('issue_action == "opened"')
df_pr_selected.query('pr_action == "opened"')

,type,created_at,repo_id,repo_name,actor_id,actor_login,pr_title,pr_number,pr_body,pr_action,...,pr_merged_by_type,pr_label,pr_review_action,pr_review_id,pr_review_body,pr_review_state,pr_review_comment_body,date,year,time_period
7655,PullRequestEvent,2016-10-19 15:54:59+00:00,18398494.0,aio-libs/aiopg,16239342.0,pyup-bot,None,188.0,None,opened,...,None,[],None,NaN,None,None,None,2016-10,2016,2016-07-01
8096,PullRequestEvent,2016-10-17 20:37:10+00:00,18398494.0,aio-libs/aiopg,16239342.0,pyup-bot,None,187.0,None,opened,...,None,[],None,NaN,None,None,None,2016-10,2016,2016-07-01
8560,PullRequestEvent,2016-10-22 06:07:06+00:00,18398494.0,aio-libs/aiopg,5688677.0,bgcgarlipp,None,190.0,None,opened,...,None,[],None,NaN,None,None,None,2016-10,2016,2016-07-01
8836,PullRequestEvent,2016-10-07 18:56:05+00:00,5399320.0,matteomattei/PySquashfsImage,395711.0,BotoX,None,4.0,None,opened,...,None,[],None,NaN,None,None,None,2016-10,2016,2016-07-01
8993,PullRequestEvent,2016-10-14 21:02:51+00:00,18398494.0,aio-libs/aiopg,16239342.0,pyup-bot,None,183.0,None,opened,...,None,[],None,NaN,None,None,None,2016-10,2016,2016-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15012,PullRequestEvent,2016-05-10 19:47:30+00:00,1424005.0,twitter/commons,1743873.0,mateor,None,428.0,None,opened,...,None,[],None,NaN,None,None,None,2016-5,2016,2016-01-01
15557,PullRequestEvent,2016-05-29 22:36:01+00:00,18398494.0,aio-libs/aiopg,857609.0,adamchainz,None,118.0,None,opened,...,None,[],None,NaN,None,None,None,2016-5,2016,2016-01-01
15767,PullRequestEvent,2016-05-19 20:52:44+00:00,50359589.0,girder/girder_worker,555959.0,zachmullen,None,36.0,None,opened,...,None,[],None,NaN,None,None,None,2016-5,2016,2016-01-01
15992,PullRequestEvent,2016-05-09 14:32:48+00:00,8858506.0,salesforce-marketingcloud/FuelSDK-Python,16882462.0,amagar088,None,61.0,None,opened,...,None,[],None,NaN,None,None,None,2016-5,2016,2016-01-01
